**NOTE**: please run this notebook from `./src`

In [ ]:
%cd ../..
%pwd

In [ ]:
from pathlib import Path
from typing import List

import pandas as pd
from pydantic import BaseModel

In [ ]:
from mock import mock  # hence the need to get the directory set up

In [ ]:
vitals_file = mock.path_to_hdf_file("perrt")
df = pd.read_hdf(vitals_file)

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
# drop na's inserted by synthetic data process
# df.dropna(subset=['id_in_application'], inplace=True)
# df.to_hdf('mock.h5', 'data')

In [ ]:
engine = mock.make_mock_db_in_memory("perrt")